In [1]:
import tensorflow as tf
# import tensorflow_decision_forests as tfdf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import pickle

2023-04-26 21:02:19.984055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X_train = pickle.load(open('./processed_facial_data/X_train.pickle', 'rb'))
y_train = pickle.load(open('./processed_facial_data/y_train.pickle', 'rb'))
X_test = pickle.load(open('./processed_facial_data/X_test.pickle', 'rb'))
y_test = pickle.load(open('./processed_facial_data/y_test.pickle', 'rb'))

In [3]:
# Normalizing data
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = X_train/255.0

X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = X_test/255.0

In [4]:
# Attempting to extract features from images through CNN t-SNE
# conv2d layers are indexed at [1, 6]
model = load_model('./facial_emotion_recognition.h5')

2023-04-26 21:02:27.475067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
layer_names = [layer.name for layer in model.layers]
layer_outputs = [layer.output for layer in model.layers]

In [6]:
feature_map_model = Model(inputs = model.input, outputs=layer_outputs)

In [7]:
new_img = X_train[0].reshape(1, X_train[0].shape[0], X_train[0].shape[1])
features = feature_map_model.predict(new_img)

1/1 [==============================] - 0s 440ms/step


In [8]:
# visualizing feature shapes
# for feature in features:
#     print(feature.shape)

In [9]:
# plt.imshow(X_test[0])
# y_test[0]

In [10]:
# visualizing the features from the cnn
# first conv2d layer has 32 filters
# second conv2d layer has 64 filters
# features are arranged as:
# features[cnn_layer_index][image_index,dimension1,dimension2,filter_index]
# hint: for filter_index, look at the amount of filters you are adding per conv2d layer (32, 64, etc)

# for i in range(len(layer_names)):
#     if layer_names[i].split('_')[0] == 'activation':
#         print(f'{i}: {layer_names[i]}')
#         try:
#             plt.imshow(features[i][0,:,:,15], cmap='Greys')
#             plt.show()
#         except:
#             print(features[i].shape)
#             continue

In [11]:
# attempting to flatten features from 'activation' layer
flattened = features[2][0,:,:,31].ravel()
# features[2][0,:,:,31].shape
# features

In [12]:
flattened.shape

(2116,)

In [13]:
# get length of features
img = X_train[0].reshape(1,X_train[0].shape[0],X_train[0].shape[1])
feature_map_model = Model(inputs = model.input, outputs=layer_outputs)
sample = feature_map_model.predict(img)

# # defining the training data matrix
# train_data = np.ndarray([X_train.shape[0],len(sample), 211], dtype=object)
# print(train_data.shape)
# print(type(train_data))
# print(train_data)
# print(train_data[0][26])

1/1 [==============================] - 0s 189ms/step


In [14]:
# features gets you a (1,dim_1,dim_2,num_filters) shaped data
# for the layers that are conv2d, we will arrannge the data as [[filter_1], [filter_2],...,[filter_n]]
# features_train[19][0]

In [15]:
# starting with the first two images of the dataset, then we can generalize onto the rest of the images
# X_train.shape[0]
for j in range(1):
    # adding an additional dimension to image (1,dim_1,dim_2)
    img = X_train[j].reshape(1, X_train[j].shape[0], X_train[j].shape[1])
    # extracting features from the images in training set
    features_train = feature_map_model.predict(img)
    print(f'FOR X_train[{j}]')
    for k in range(len(features_train)):
        conv_features = []
        dense_features = []
        if len(features_train[k].shape) == 4:       # extract features from conv2d layers
            for l in range(features_train[k].shape[3]):  # append all conv2d filters (32 or 64 filters) to our conv_features array
                conv_features.append( features_train[k][0,:,:,l].ravel() )

        elif len(features_train[k].shape) == 2:     # extract features from dense layers
            dense_features.append(features_train[k][0])
        
        
        if len(conv_features) != 0:
            conv_features = np.array(conv_features)
            print(f'conv_features length = {len(conv_features)} with conv_features.shape = {conv_features.shape}, total_parameters = {conv_features.shape[0]*conv_features.shape[1]}')
        
        if len(dense_features) != 0:
            dense_features = np.array(dense_features)
            print(f'dense_features length = {len(dense_features)} with dense_features.shape = {dense_features.shape}')
    print('')

1/1 [==============================] - 0s 18ms/step
FOR X_train[0]
conv_features length = 32 with conv_features.shape = (32, 2116), total_parameters = 67712
conv_features length = 32 with conv_features.shape = (32, 2116), total_parameters = 67712
conv_features length = 32 with conv_features.shape = (32, 2116), total_parameters = 67712
conv_features length = 32 with conv_features.shape = (32, 529), total_parameters = 16928
conv_features length = 32 with conv_features.shape = (32, 529), total_parameters = 16928
conv_features length = 64 with conv_features.shape = (64, 529), total_parameters = 33856
conv_features length = 64 with conv_features.shape = (64, 529), total_parameters = 33856
conv_features length = 64 with conv_features.shape = (64, 529), total_parameters = 33856
conv_features length = 64 with conv_features.shape = (64, 121), total_parameters = 7744
conv_features length = 64 with conv_features.shape = (64, 121), total_parameters = 7744
conv_features length = 64 with conv_featur

In [16]:
# concluding from the above printing, we can easily now picture how our dataset would look like. 
# We will have 27 different datasets, and we will store them into an array that will have the following shape:
# train_data = [conv_feature_1, conv_feature_2, ..., conv_feature_n, dense_feature_1, ..., dense_feature_m]
# where the features of the layers would look like this:
# conv_feature_x ----> length = num_filters
#                ----> array_entry = 1-D array of images
#                ----> each filter will have the same label 
# dense_feature_x ---> length = num_neurons


train_data = []
for j in range(X_train.shape[0]):
    # adding an additional dimension to image (1,dim_1,dim_2)
    img = X_train[j].reshape(1, X_train[j].shape[0], X_train[j].shape[1])
    # extracting features from the images in training set
    features_train = feature_map_model.predict(img)
    for k in range(len(features_train)):
        if len(features_train[k].shape) == 4:       # extract features from conv2d layers
            conv_features = []
            for l in range(features_train[k].shape[3]):  # append all conv2d filters (32 or 64 filters) to our conv_features array
                conv_features.append( features_train[k][0,:,:,l].ravel() )
            # flattening filter vectors
            conv_features = np.array(conv_features).ravel()
            # storing our feature vectors
            train_data.append( (conv_features, y_train[j]) )

        elif len(features_train[k].shape) == 2:     # extract features from dense layers
            dense_features = features_train[k][0]
            train_data.append( (dense_features, y_train[j]) )
                

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step


...

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 18ms/step


In [17]:
train_data[0]

(array([-0.03288623, -0.01935673, -0.01263739, ...,  0.00850673,
         0.00559484,  0.00593166], dtype=float32),
 5)

In [18]:
# for i in range(0,1000,27):
#     print(f'{i}.) train_data_shape = {train_data[i][0].shape}, label = {train_data[i][1]}', end='\n\n')
#     print(train_data[i][1],end='\n\n')


In [19]:
# WARNING: DO NOT run this cell unless you want to kill your kernel!!! Data is too big (7GB) :)
# np_train_data = np.array(train_data)
# np.save('GBD_train_data',np_train_data)

In [20]:
def create_datasets(data, j, layer_names):
    # next create a function that sepaarates the data into datasets per layer
    # i will keep track of the indices of train_data and be updated by +27
    # count will keep track of the amout of datasets alredy created (and will also keep track of layer_names index)
    i = j
    
    count = 0
    output_data = []
    output_labels = []
    while i < len(train_data):
        # will try to save dataset with layer_name files
        output_data.append( train_data[i][0] )
        output_labels.append( train_data[i][1] )
        i+=27
    file_data = f'./GBT_data/train/data/{layer_names[j]}_layer_{j}_data'
    file_labels = f'./GBT_data/train/labels/{layer_names[j]}_layer_{j}_labels'
    np.save(file_data, output_data)
    np.save(file_labels, output_data)

In [1]:
for j in range(len(layer_names)):
    create_datasets(train_data, j, layer_names)

NameError: name 'layer_names' is not defined

In [ ]:
len(train_data)/27

In [ ]:
# x = 28000x64 and y = 28000x1
# tree_model = tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.CLASSIFICATION)
# tree_model.fit(train_data[:100][0], )

In [ ]:
# tree_model = tfdf.keras.GradientBoostedTreesModel()
# tree_model.fit(tf_dataset)